In [1]:
#Mounts Google Colab on Gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
 %cd /content/gdrive/My Drive
#Create our working directory
%mkdir HUrUAI

%cd /content/gdrive/My Drive/HUrUAI
%mkdir CatDogClass

# Move into the working directory
%cd /content/gdrive/My Drive/HUrUAI

/content/gdrive/My Drive
mkdir: cannot create directory ‘HUrUAI’: File exists
/content/gdrive/My Drive/HUrUAI
mkdir: cannot create directory ‘CatDogClass’: File exists
/content/gdrive/My Drive/HUrUAI


Install the required packages

In [3]:
!pip install split-folders

Import the packages to use

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import the required packages

import numpy as np
import os

import torch 
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import Image

import matplotlib.pyplot as plt
import splitfolders

Check for GPU availability

In [5]:
#check if CUDA is available
train_on_gpu = torch.cuda.is_available

if not train_on_gpu:
    print('CUDA is not available')
else:
    print('CUDA is available! Training on GPU...')

CUDA is available! Training on GPU...


Preparing the Dataset

Specify your path to the kaggle dir containing the kaggle.json file

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/HUrUAI/CatDogClass/kaggle/"

Download the Dataset

In [7]:
!kaggle datasets download -d erkamk/cat-and-dog-images-dataset

 92% 118M/128M [00:01<00:00, 61.4MB/s]
100% 128M/128M [00:01<00:00, 69.7MB/s]


In [8]:
#This code unzips the compressed file received from kaggle.

!unzip \*.zip && rm *.zip

Archive:  cat-and-dog-images-dataset.zip
  inflating: Dog and Cat .png/Cat/1.png  
  inflating: Dog and Cat .png/Cat/10.png  
  inflating: Dog and Cat .png/Cat/100.png  
  inflating: Dog and Cat .png/Cat/101.png  
  inflating: Dog and Cat .png/Cat/102.png  
  inflating: Dog and Cat .png/Cat/103.png  
  inflating: Dog and Cat .png/Cat/104.png  
  inflating: Dog and Cat .png/Cat/105.png  
  inflating: Dog and Cat .png/Cat/106.png  
  inflating: Dog and Cat .png/Cat/107.png  
  inflating: Dog and Cat .png/Cat/108.png  
  inflating: Dog and Cat .png/Cat/109.png  
  inflating: Dog and Cat .png/Cat/11.png  
  inflating: Dog and Cat .png/Cat/110.png  
  inflating: Dog and Cat .png/Cat/111.png  
  inflating: Dog and Cat .png/Cat/112.png  
  inflating: Dog and Cat .png/Cat/113.png  
  inflating: Dog and Cat .png/Cat/114.png  
  inflating: Dog and Cat .png/Cat/115.png  
  inflating: Dog and Cat .png/Cat/116.png  
  inflating: Dog and Cat .png/Cat/117.png  
  inflating: Dog and Cat .png/Cat/118.p


Load the Data

In [13]:
data_dir = './Data'

In [14]:
splitfolders.ratio(data_dir, output="SplitData", ratio=(.7, .15, .15))




Copying files: 0 files [00:00, ? files/s]


Copying files: 4 files [00:00, 39.33 files/s]


Copying files: 15 files [00:00, 48.63 files/s]


Copying files: 24 files [00:00, 51.61 files/s]


Copying files: 38 files [00:00, 63.57 files/s]


Copying files: 46 files [00:00, 64.43 files/s]


Copying files: 59 files [00:00, 75.54 files/s]


Copying files: 68 files [00:00, 73.70 files/s]


Copying files: 78 files [00:00, 72.35 files/s]


Copying files: 93 files [00:01, 84.55 files/s]


Copying files: 103 files [00:01, 65.05 files/s]


Copying files: 114 files [00:01, 68.56 files/s]


Copying files: 122 files [00:01, 54.72 files/s]


Copying files: 129 files [00:01, 46.61 files/s]


Copying files: 141 files [00:02, 52.69 files/s]


Copying files: 154 files [00:02, 63.70 files/s]


Copying files: 163 files [00:02, 64.85 files/s]


Copying files: 174 files [00:02, 65.30 files/s]


Copying files: 188 files [00:02, 77.44 files/s]


Copying files: 198 files [00:02, 74.44 files/s]


Copying files

In [15]:
new_data_dir = './SplitData'
train_dir = new_data_dir + '/train'
val_dir = new_data_dir + '/val'
test_dir = new_data_dir + '/test'

Define your transforms

In [16]:
#To Do
#Define your train, validation and test transforms

data_transforms = { 
            'train': transforms.Compose([
                          
    transforms.RandomRotation(5) ,
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),                                                        
    transforms.ToTensor(
    ),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
]),

             'val': transforms.Compose([
                                        
    transforms.RandomRotation(5) ,
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),                                                                   
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
]),

             'test': transforms.Compose([
                                         
    transforms.RandomRotation(5) ,
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),                                                                
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])])}

Implement a function to check the validity of the files

In [17]:
def check_image(path):
  try:
    im = Image.open(path)
    return True
  except:
    return False

Instantiate Our Classes


In [18]:
classes = ['Cat', 'Dog']

Create Our Train, Validation and Test Dataset

In [19]:
# To Do

# Create your Train, validation and test dataset objects.
train_data = datasets.ImageFolder(data_dir, transform=data_transforms)
val_data = datasets.ImageFolder(data_dir, transform=data_transforms)
test_data = datasets.ImageFolder(data_dir, transform=data_transforms)

Define Our Data Loader

In [20]:
# TO Do

#Specify values for your Batch size and num_workers.
batch_size = 20
num_workers = 0

## Create your train, validation and test data loaders.

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_dir)
valid_sampler = SubsetRandomSampler(val_dir)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

Building Our Model

In [21]:
# Load the pretrained model from pytorch
model = models.vgg16(pretrained=True)

# print out the model structure
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [22]:
# TO Do

# Freeze training for all the hidden layers of the model   
# Freeze training for all "features" layers
for param in model.features.parameters():
    param.requires_grad = False

In [23]:
# To Do

# Modify the output layer of your model to match the number of classes you have
import torch.nn as nn

n_inputs = model.classifier[6].in_features

# add last linear layer (n_inputs -> 5 flower classes)
# new layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, len(classes))

model.classifier[6] = last_layer

# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()

# check to see that your last layer produces the expected number of outputs
print(model.classifier[6].out_features)
print(model)

2
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

Specify and Loss Function

In [24]:
# TO DO

# specify loss function
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001)

Train Your Network

In [25]:
# TO Do

# number of epochs to train the model
n_epochs = 5
  
valid_loss_min = np.Inf # track change in validation loss

# Initialize two lists to store our training and validation losses.
train_losses, valid_losses = [0.0], [0.0]

for epoch in range(1, n_epochs+1):

    
    ###################
    # train the model #
    ###################
    
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # To Do: clear the gradients of all optimized variables
        optimizer.zero_grad()
        # To Do: forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # To Do: calculate the batch loss
        loss = criterion(output, target)
        # To Do: backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # To Do: perform a single optimization step (parameter update)
        optimizer.step()
        # To Do: update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # To Do: move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # To Do: forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # To Do: calculate the batch loss
        loss = criterion(output, target)
        # To Do: update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # To Do: calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)


    # append train_loss and valid_loss to the train_losses and valid_losses for visualization
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        # To Do: Save your model to drive anytime the validation loss goes down
        torch.save(model.state-dict(), 'model_cat_and_dog.pt')
        valid_loss_min = valid_loss

TypeError: ignored

In [ ]:
# Plot the training and validation losses. This will help you keep track of your training process and identify the early
# stopping point, hence avoiding overfitting the model to the data.

plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.legend(frameon=False)

Testing the Network

In [ ]:
#To Do: Load your saved model from GDrive for testing
model.load_state_dict(torch.load('model_cat_and_dog.pt'))

Test the Model on Test set

In [ ]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


In [ ]:
test_model(model)